In [1]:
import gensim
import numpy as np
import pandas as pd
import h5py
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
import re
from gensim.models import Word2Vec, FastText
from gensim.models.word2vec import LineSentence
from joblib import Parallel, delayed 


/home/data/khuangaf/miniconda2/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
regex = re.compile('[%s]' % re.escape(string.punctuation))

train_text = pd.read_csv('../input/train.csv').comment_text.apply(lambda x: regex.sub(' ', x).replace('\n', ' ').split())
test_text = pd.read_csv('../input/test.csv').comment_text.apply(lambda x: regex.sub(' ', x).replace('\n', ' ').split() if isinstance(x, basestring) else 'Steeve')

In [43]:
concated = train_text.values.tolist() + test_text.values.tolist()
concated

[['Explanation',
  'Why',
  'the',
  'edits',
  'made',
  'under',
  'my',
  'username',
  'Hardcore',
  'Metallica',
  'Fan',
  'were',
  'reverted',
  'They',
  'weren',
  't',
  'vandalisms',
  'just',
  'closure',
  'on',
  'some',
  'GAs',
  'after',
  'I',
  'voted',
  'at',
  'New',
  'York',
  'Dolls',
  'FAC',
  'And',
  'please',
  'don',
  't',
  'remove',
  'the',
  'template',
  'from',
  'the',
  'talk',
  'page',
  'since',
  'I',
  'm',
  'retired',
  'now',
  '89',
  '205',
  '38',
  '27'],
 ['D',
  'aww',
  'He',
  'matches',
  'this',
  'background',
  'colour',
  'I',
  'm',
  'seemingly',
  'stuck',
  'with',
  'Thanks',
  'talk',
  '21',
  '51',
  'January',
  '11',
  '2016',
  'UTC'],
 ['Hey',
  'man',
  'I',
  'm',
  'really',
  'not',
  'trying',
  'to',
  'edit',
  'war',
  'It',
  's',
  'just',
  'that',
  'this',
  'guy',
  'is',
  'constantly',
  'removing',
  'relevant',
  'information',
  'and',
  'talking',
  'to',
  'me',
  'through',
  'edits',
  'ins

In [56]:
train = pd.concat([pd.read_csv('../input/train.csv'),pd.read_csv('../input/train_de.csv'),pd.read_csv('../input/train_es.csv'),pd.read_csv('../input/train_fr.csv')])
test = pd.read_csv('../input/test.csv')
train['comment_text'] = train['comment_text'].fillna('_empty_')
test['comment_text'] = test['comment_text'].fillna('_empty_')

In [6]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train['comment_text'] = train['comment_text'].fillna('_empty_')
test['comment_text'] = test['comment_text'].fillna('_empty_')

In [7]:
# PREPROCESSING PART
repl = {
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}
keys = [i for i in repl.keys()]

In [5]:
new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()

In [6]:
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)

In [7]:
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)

In [10]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [11]:
train["comment_text"] = new_train_data
test["comment_text"] = new_test_data
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d'aww! he matches this background colour i am ...,0,0,0,0,0,0
2,000113f07ec002fd,"hey man, i am really not trying to edit war. i...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" more i can not make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0


In [8]:
import re
import numpy as np
import pandas as pd
from fastText import load_model



def normalize(s):
    """
    Given a text, cleans and normalizes it. Feel free to add your own stuff.
    """
    s = s.lower()
    # Replace ips
    s = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', s)
    # Isolate punctuation
    s = re.sub(r'([\'\"\.\(\)\!\?\-\\\/\,])', r' \1 ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Replace numbers and symbols with language
    s = s.replace('&', ' and ')
    s = s.replace('@', ' at ')
    s = s.replace('0', ' zero ')
    s = s.replace('1', ' one ')
    s = s.replace('2', ' two ')
    s = s.replace('3', ' three ')
    s = s.replace('4', ' four ')
    s = s.replace('5', ' five ')
    s = s.replace('6', ' six ')
    s = s.replace('7', ' seven ')
    s = s.replace('8', ' eight ')
    s = s.replace('9', ' nine ')
    return s




In [9]:
print('\nLoading FT model')
ft_model = load_model('../wiki.en.bin')
n_features = ft_model.get_dimension()
print (n_features)


Loading FT model
300


In [10]:
classes = [
    'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'
]
from nltk.tokenize import word_tokenize
import re

window_length = 300 # The amount of words we look at per example. Experiment with this.
def text_to_vector(text):
    """
    Given a string, normalizes it, then splits it into words and finally converts
    it to a sequence of word vectors.
    """

    tokens = word_tokenize(text.decode("utf8"))
    tokens = [w.lower() for w in tokens]
    stripped = [regex.sub('', t).replace('\n', '') for t in tokens]
    # remove remaining tokens that are not alphabetic
    
    words = [word for word in stripped if word.isalpha()]
#     from nltk.corpus import stopwords
#     stop_words = set(stopwords.words('english'))
#     words = [w for w in words if not w in stop_words]
    text = ' '.join(words)
    text = normalize(text)
    words = text.split()
    window = words[-window_length:]
    
    x = np.zeros((window_length, n_features))

    for i, word in enumerate(window):
        x[i, :] = ft_model.get_word_vector(word).astype('float32')

    return x

def df_to_data(df):
    """
    Convert a given dataframe to a dataset of inputs for the NN.
    """
    x = np.zeros((len(df), window_length, n_features), dtype='float32')
#     parallel = Parallel(n_jobs=10, backend="threading", verbose=5)
#     translated_data = parallel(delayed(translate)(comment, language) for comment in comments_list)
    for i, comment in enumerate(df['comment_text'].values):
        x[i, :] = text_to_vector(comment)
#     x = parallel(delayed(text_to_vector)(comment) for comment in df['comment_text'].values)
#     x = np.array(x)
    return x

In [11]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
x_train = df_to_data(train)
y_train = train[classes].values

x_test = df_to_data(test)
#y_test = test[classes].values

In [12]:
import h5py

#ct: use preprocessing to clean texts (emoji ...)
#rmnum: remove numeric value
with h5py.File('../input/fasttext300_processed_rmnum.h5','w') as f:
    f.create_dataset("x_train", data = x_train )
    f.create_dataset("x_test", data = x_test )
    f.create_dataset("y_train", data = y_train )
#     f.create_dataset("embedding_matrix", data=embedding_matrix)
    

In [13]:
x_train.shape

(159571, 300, 300)

In [3]:
with h5py.File('../input/fasttext_processed_rmnum_125.h5','r') as f:
    x_train = f['x_train'].value

In [4]:
x_train.shape

(159571, 100, 300)